In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import folium

warnings.filterwarnings(action='ignore')
#pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.options.display.float_format="{:.5f}".format
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [2]:
data = pd.read_csv('data/data.csv')
#submission = pd.read_csv('data/submission.csv')
target = pd.read_csv('data/target.csv')
#student = pd.read_csv('data/인천_학령인구_추계.csv', encoding = 'cp949')

In [3]:
data.head()

,수요지 id,X,Y,초등학령인구
0,C1,126.37150,37.46378,1
1,C2,126.47950,37.49578,46
2,C3,126.48150,37.49978,36
3,C4,126.48250,37.49578,81
4,C5,126.48350,37.49778,25


In [4]:
data.columns = ['수요지 id', '수요_X', '수요_Y', '초등학령인구']

In [5]:
data.head()

,수요지 id,수요_X,수요_Y,초등학령인구
0,C1,126.37150,37.46378,1
1,C2,126.47950,37.49578,46
2,C3,126.48150,37.49978,36
3,C4,126.48250,37.49578,81
4,C5,126.48350,37.49778,25


In [6]:
target.head()

,학교 id,X,Y
0,F1,126.54392,37.46729
1,F2,126.56873,37.49630
2,F3,126.49277,37.48443
3,F4,126.62968,37.49578
4,F5,126.52829,37.45028


In [7]:
target.columns = ['학교 id', '공급_X', '공급_Y']

In [8]:
target.head()

,학교 id,공급_X,공급_Y
0,F1,126.54392,37.46729
1,F2,126.56873,37.49630
2,F3,126.49277,37.48443
3,F4,126.62968,37.49578
4,F5,126.52829,37.45028


In [9]:
sum_df = pd.DataFrame()

# 반복문을 통해 df1 각 행에 df2를 붙이기
for _, row in target.iterrows():
    temp_df = pd.concat([pd.DataFrame(row).T] * len(data), ignore_index=True)
    temp_df = temp_df.join(data)
    sum_df = pd.concat([sum_df, temp_df])

sum_df

,학교 id,공급_X,공급_Y,수요지 id,수요_X,수요_Y,초등학령인구
0,F1,126.54392,37.46729,C1,126.37150,37.46378,1
1,F1,126.54392,37.46729,C2,126.47950,37.49578,46
2,F1,126.54392,37.46729,C3,126.48150,37.49978,36
3,F1,126.54392,37.46729,C4,126.48250,37.49578,81
4,F1,126.54392,37.46729,C5,126.48350,37.49778,25
...,...,...,...,...,...,...,...
306,F50,126.50114,37.51461,C307,126.65150,37.48278,2
307,F50,126.50114,37.51461,C308,126.65250,37.47278,12
308,F50,126.50114,37.51461,C309,126.65250,37.47978,1
309,F50,126.50114,37.51461,C310,126.65250,37.48078,1


In [10]:
sum_df = sum_df.drop('초등학령인구', axis = 1)

In [11]:
sum_df['위도_범위'] = sum_df.apply(lambda row: sorted([row['수요_Y'], row['공급_Y']]), axis=1)

In [12]:
sum_df['경도_범위'] = sum_df.apply(lambda row: sorted([row['수요_X'], row['공급_X']]), axis=1)

In [13]:
sum_df.head()

,학교 id,공급_X,공급_Y,수요지 id,수요_X,수요_Y,위도_범위,경도_범위
0,F1,126.54392,37.46729,C1,126.37150,37.46378,"[37.4637824, 37.46729212308389]","[126.3715048, 126.54392107334832]"
1,F1,126.54392,37.46729,C2,126.47950,37.49578,"[37.46729212308389, 37.4957824]","[126.4795048, 126.54392107334832]"
2,F1,126.54392,37.46729,C3,126.48150,37.49978,"[37.46729212308389, 37.4997824]","[126.4815048, 126.54392107334832]"
3,F1,126.54392,37.46729,C4,126.48250,37.49578,"[37.46729212308389, 37.4957824]","[126.4825048, 126.54392107334832]"
4,F1,126.54392,37.46729,C5,126.48350,37.49778,"[37.46729212308389, 37.4977824]","[126.4835048, 126.54392107334832]"


In [14]:
sum_df['버스정류장_개수'] = 0

In [15]:
sum_df

,학교 id,공급_X,공급_Y,수요지 id,수요_X,수요_Y,위도_범위,경도_범위,버스정류장_개수
0,F1,126.54392,37.46729,C1,126.37150,37.46378,"[37.4637824, 37.46729212308389]","[126.3715048, 126.54392107334832]",0
1,F1,126.54392,37.46729,C2,126.47950,37.49578,"[37.46729212308389, 37.4957824]","[126.4795048, 126.54392107334832]",0
2,F1,126.54392,37.46729,C3,126.48150,37.49978,"[37.46729212308389, 37.4997824]","[126.4815048, 126.54392107334832]",0
3,F1,126.54392,37.46729,C4,126.48250,37.49578,"[37.46729212308389, 37.4957824]","[126.4825048, 126.54392107334832]",0
4,F1,126.54392,37.46729,C5,126.48350,37.49778,"[37.46729212308389, 37.4977824]","[126.4835048, 126.54392107334832]",0
...,...,...,...,...,...,...,...,...,...
306,F50,126.50114,37.51461,C307,126.65150,37.48278,"[37.4827824, 37.514612976919096]","[126.5011352571878, 126.6515048]",0
307,F50,126.50114,37.51461,C308,126.65250,37.47278,"[37.4727824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0
308,F50,126.50114,37.51461,C309,126.65250,37.47978,"[37.4797824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0
309,F50,126.50114,37.51461,C310,126.65250,37.48078,"[37.4807824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0


In [16]:
sum_df.reset_index(drop=True, inplace=True)

In [17]:
sum_df

,학교 id,공급_X,공급_Y,수요지 id,수요_X,수요_Y,위도_범위,경도_범위,버스정류장_개수
0,F1,126.54392,37.46729,C1,126.37150,37.46378,"[37.4637824, 37.46729212308389]","[126.3715048, 126.54392107334832]",0
1,F1,126.54392,37.46729,C2,126.47950,37.49578,"[37.46729212308389, 37.4957824]","[126.4795048, 126.54392107334832]",0
2,F1,126.54392,37.46729,C3,126.48150,37.49978,"[37.46729212308389, 37.4997824]","[126.4815048, 126.54392107334832]",0
3,F1,126.54392,37.46729,C4,126.48250,37.49578,"[37.46729212308389, 37.4957824]","[126.4825048, 126.54392107334832]",0
4,F1,126.54392,37.46729,C5,126.48350,37.49778,"[37.46729212308389, 37.4977824]","[126.4835048, 126.54392107334832]",0
...,...,...,...,...,...,...,...,...,...
15545,F50,126.50114,37.51461,C307,126.65150,37.48278,"[37.4827824, 37.514612976919096]","[126.5011352571878, 126.6515048]",0
15546,F50,126.50114,37.51461,C308,126.65250,37.47278,"[37.4727824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0
15547,F50,126.50114,37.51461,C309,126.65250,37.47978,"[37.4797824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0
15548,F50,126.50114,37.51461,C310,126.65250,37.48078,"[37.4807824, 37.514612976919096]","[126.5011352571878, 126.6525048]",0


In [18]:
warning = pd.read_csv('data/인천_버스정류장_위경도.csv', encoding = 'cp949')

In [19]:
warning.head()

,1번째 칼럼,2번째 칼럼,3번째 칼럼,4번째 칼럼,5번째 칼럼,6번째 칼럼,입력주소,X,Y,CLSS,PNU,주소구분,표준신주소,표준구주소,우편번호,행정동코드,행정동명,법정동코드,법정동명
0,2441,동구,만석초등학교,인천광역시 동구 제물량로 413 버스정류장,사물주소 확정,2021-11-25,인천광역시 동구 제물량로 413 버스정류장,126.62708,37.48444,인근좌표,28140.00000,새주소,인천광역시 동구 제물량로 413,인천광역시 동구,NaN,NaN,NaN,NaN,NaN
1,2442,동구,창영어린이공원,인천광역시 동구 우각로 30 버스정류장,사물주소 확정,2021-09-17,인천광역시 동구 우각로 30 버스정류장,126.63944,37.47080,인근좌표,28140.00000,새주소,인천광역시 동구 우각로 30,인천광역시 동구,NaN,NaN,NaN,NaN,NaN
2,2443,동구,참좋은집,인천광역시 동구 방축로62번길 37 버스정류장,사물주소 확정,2021-09-17,인천광역시 동구 방축로62번길 37 버스정류장,126.65217,37.48319,정좌표,2814010000000000000.00000,새주소,인천광역시 동구 방축로62번길 37,인천광역시 동구 송림동 8-26,22529.00000,2814063000.00000,송림6동,2814010700.00000,송림동
3,2444,동구,참좋은집,인천광역시 동구 방축로62번길 38 버스정류장,사물주소 확정,2021-09-17,인천광역시 동구 방축로62번길 38 버스정류장,126.65139,37.48388,정좌표,2814010000000000000.00000,새주소,인천광역시 동구 방축로62번길 38,인천광역시 동구 송림동 8-810,22529.00000,2814063000.00000,송림6동,2814010700.00000,송림동
4,2445,동구,인천재능대후문,인천광역시 동구 재능로 167 버스정류장,사물주소 확정,2021-09-17,인천광역시 동구 재능로 167 버스정류장,126.64965,37.47628,인근좌표,28140.00000,새주소,인천광역시 동구 재능로 167,인천광역시 동구,NaN,NaN,NaN,NaN,NaN


In [20]:
from tqdm import tqdm
import time

In [21]:
for i in tqdm(range(len(sum_df))):
    time.sleep(0.01)
    for j in range(len(warning)):
        if (sum_df.iloc[i, 6][0] <= warning.iloc[j,8] <= sum_df.iloc[i, 6][1]) and (sum_df.iloc[i, 7][0] <= warning.iloc[j,7] <= sum_df.iloc[i, 7][1]):
            sum_df.at[i, '버스정류장_개수'] += 1

100%|████████████████████████████████████████████████████████████████████████████| 15550/15550 [11:48<00:00, 21.94it/s]


In [22]:
sum_df

,학교 id,공급_X,공급_Y,수요지 id,수요_X,수요_Y,위도_범위,경도_범위,버스정류장_개수
0,F1,126.54392,37.46729,C1,126.37150,37.46378,"[37.4637824, 37.46729212308389]","[126.3715048, 126.54392107334832]",4
1,F1,126.54392,37.46729,C2,126.47950,37.49578,"[37.46729212308389, 37.4957824]","[126.4795048, 126.54392107334832]",88
2,F1,126.54392,37.46729,C3,126.48150,37.49978,"[37.46729212308389, 37.4997824]","[126.4815048, 126.54392107334832]",100
3,F1,126.54392,37.46729,C4,126.48250,37.49578,"[37.46729212308389, 37.4957824]","[126.4825048, 126.54392107334832]",86
4,F1,126.54392,37.46729,C5,126.48350,37.49778,"[37.46729212308389, 37.4977824]","[126.4835048, 126.54392107334832]",88
...,...,...,...,...,...,...,...,...,...
15545,F50,126.50114,37.51461,C307,126.65150,37.48278,"[37.4827824, 37.514612976919096]","[126.5011352571878, 126.6515048]",224
15546,F50,126.50114,37.51461,C308,126.65250,37.47278,"[37.4727824, 37.514612976919096]","[126.5011352571878, 126.6525048]",364
15547,F50,126.50114,37.51461,C309,126.65250,37.47978,"[37.4797824, 37.514612976919096]","[126.5011352571878, 126.6525048]",266
15548,F50,126.50114,37.51461,C310,126.65250,37.48078,"[37.4807824, 37.514612976919096]","[126.5011352571878, 126.6525048]",248


In [23]:
sum_df['버스정류장_개수'].value_counts()

0      652
2      477
4      361
1      301
6      286
      ... 
371      1
319      1
356      1
352      1
364      1
Name: 버스정류장_개수, Length: 374, dtype: int64

In [24]:
sum_df.to_csv('버스정류장_개수_weight.csv', index=False)